# Imports

In [29]:
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.regularizers import l2, activity_l2

In [2]:
np.random.seed(1)

# Loading MNIST

In [3]:
from keras.datasets import mnist

(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [4]:
print X_train.shape

(60000, 28, 28)


# Preprocessing data

Reshaping from (n, width, height) to (n, depth, width, height), where depth is the number of image channels. In a RGB image, it is 3 -- one for each color -- but for this data, all images are in grayscale, so depth=1.

In [5]:
X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1], X_train.shape[2])
X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1], X_test.shape[2])

In [6]:
print X_train.shape

(60000, 1, 28, 28)


In [7]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255.
X_test /= 255.

### Preprocessing labels - one-hot representation

In [8]:
print y_train.shape

(60000,)


In [9]:
Y_train = np_utils.to_categorical(y_train, 10)
Y_test = np_utils.to_categorical(y_test, 10)

In [10]:
print Y_train.shape

(60000, 10)


# Model

In [57]:
model = Sequential()
 
model.add(Convolution2D(32, 3, 3, 
                        activation='relu', 
                        input_shape=(1, 28, 28), 
                        dim_ordering='th', 
#                         W_regularizer=l2(1.),
#                         activity_regularizer=activity_l2(1.)
                       )
         )
print model.output_shape

(None, 32, 26, 26)


In [58]:
model.add(Convolution2D(32, 3, 3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
 
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

In [59]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [60]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_15 (Convolution2D) (None, 32, 26, 26)    320         convolution2d_input_9[0][0]      
____________________________________________________________________________________________________
convolution2d_16 (Convolution2D) (None, 30, 24, 32)    7520        convolution2d_15[0][0]           
____________________________________________________________________________________________________
maxpooling2d_7 (MaxPooling2D)    (None, 15, 12, 32)    0           convolution2d_16[0][0]           
____________________________________________________________________________________________________
dropout_13 (Dropout)             (None, 15, 12, 32)    0           maxpooling2d_7[0][0]             
___________________________________________________________________________________________

# Fit

In [63]:
checkpoint = ModelCheckpoint('weights.{epoch:02d}-{val_loss:.2f}.hdf5',
                             monitor='val_loss',
                             verbose=0,
                             save_best_only=False,
                             save_weights_only=True
                            )

early_stopping = EarlyStopping(monitor='val_loss', patience=2)

training_history = model.fit(X_train, 
                             Y_train, 
                             batch_size=32, 
                             nb_epoch=100,
                             validation_data=(X_test, Y_test),
                             callbacks=[checkpoint, early_stopping],
                             verbose=2
                            )

Train on 60000 samples, validate on 10000 samples
Epoch 1/100
16s - loss: 0.1577 - acc: 0.9554 - val_loss: 0.0508 - val_acc: 0.9829
Epoch 2/100
16s - loss: 0.0933 - acc: 0.9724 - val_loss: 0.0439 - val_acc: 0.9857
Epoch 3/100
17s - loss: 0.0738 - acc: 0.9783 - val_loss: 0.0389 - val_acc: 0.9880
Epoch 4/100
18s - loss: 0.0629 - acc: 0.9812 - val_loss: 0.0336 - val_acc: 0.9899
Epoch 5/100
18s - loss: 0.0575 - acc: 0.9828 - val_loss: 0.0329 - val_acc: 0.9893
Epoch 6/100
18s - loss: 0.0484 - acc: 0.9848 - val_loss: 0.0335 - val_acc: 0.9891
Epoch 7/100
17s - loss: 0.0457 - acc: 0.9860 - val_loss: 0.0318 - val_acc: 0.9892
Epoch 8/100
18s - loss: 0.0393 - acc: 0.9879 - val_loss: 0.0291 - val_acc: 0.9908
Epoch 9/100
18s - loss: 0.0381 - acc: 0.9883 - val_loss: 0.0267 - val_acc: 0.9912
Epoch 10/100
18s - loss: 0.0350 - acc: 0.9896 - val_loss: 0.0299 - val_acc: 0.9910
Epoch 11/100
18s - loss: 0.0317 - acc: 0.9900 - val_loss: 0.0367 - val_acc: 0.9897
Epoch 12/100
18s - loss: 0.0288 - acc: 0.9908 